# INSTRUCTION TO RUN THE CODE:



1.   **Training and Testing**



*   **Loading Dataset** : Load the dataset for training and give path of the dataset at 'Data Processing block' for reading the dataset.
*   **Testing Dataset** : For testing yor csv file you have to load your test_dataset.csv file and provide its path to the block 'preprocessing of test Dataset'.


*   For removing stopword you have to upload one extra file final_stopwords.tx and provide its correct path



2.   **Loading trained Model and Test:**


*   You can test your test csv by just loading the already saved trained model lstm_model.pth that reduce training time


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installing Neccesary library

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import locale
print(locale.getdefaultlocale())
print(locale.getpreferredencoding())

('en_US', 'UTF-8')
UTF-8


In [ ]:
!pip install torchtext
!pip install torchtext==0.10.0
!pip install Pytorch
!pip install emoji
!pip install nltk
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 2.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.9.0 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0,

# Import require Library

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

import collections
from collections import Counter



# Trainig Data Preprocessing

In [ ]:
# Loading the dataset
full_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/hindi_train_val.csv')
# df = full_df[["text"]]
df = full_df
# df["text"] = df["text"].astype(str)
full_df.head()

,label,text
0,0,भीड़ में बहुत हब्सी मिलेंगे
1,0,साले बेवकूफ अपनी मां मक्खियां तो हटा दें
2,0,बुर देदो तो मुह में लंड ले लो तो
3,0,कुत्ता वहा है चिल्ला तू क्यों रहा है
4,1,चाय नहीं पीता हूं मैं इसी को छोड़ दिया ok


In [ ]:
!pip install indic_transliteration indicnlp indic-nlp-library indic


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 17.5 MB/s eta 0:00:00
  Created wheel for indic: filename=indic-0.1.2-py3-none-any.whl size=10089 sha256=77d6979e12d24ad52dd8075948f02b1eeba5f4cac197a3dfad7dc8622c7164e6
  Stored in directory: /root/.cache/pip/wheels/99/46/ae/5111371e0c7d7160c689f9f6cee321acd45a4e32d4c419dd4a
Successfully built indic


In [ ]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [ ]:
# stopword list

with open('/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/final_stopwords.txt', 'r', encoding="utf8") as file:
    content = file.readlines()

stopwords_hi = [line.strip() for line in content]

# stopwords_hi = ['तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','आगे','माननीय','शहर','बताएं','कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें','आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर', 'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'तो', 'था', 'थी', 'थे', 'दबारा', 'दिया', 'दुसरा', 'दूसरे', 'दो', 'द्वारा', 'न', 'नहीं', 'ना', 'निहायत', 'नीचे', 'ने', 'पर', 'पर', 'पहले', 'पूरा', 'पे', 'फिर', 'बनी', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल', 'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे', 'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वर्ग', 'वह', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'वग़ैरह', 'संग', 'सकता', 'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'ही', 'हुआ', 'हुई', 'हुए', 'है', 'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने', 'अपनि', 'जेसे', 'होति', 'सभि', 'तिंहों', 'इंहों', 'दवारा', 'इसि', 'किंहें', 'थि', 'उंहों', 'ओर', 'जिंहें', 'वहिं', 'अभि', 'बनि', 'हि', 'उंहिं', 'उंहें', 'हें', 'वगेरह', 'एसे', 'रवासा', 'कोन', 'निचे', 'काफि', 'उसि', 'पुरा', 'भितर', 'हे', 'बहि', 'वहां', 'कोइ', 'यहां', 'जिंहों', 'तिंहें', 'किसि', 'कइ', 'यहि', 'इंहिं', 'जिधर', 'इंहें', 'अदि', 'इतयादि', 'हुइ', 'कोनसा', 'इसकि', 'दुसरे', 'जहां', 'अप', 'किंहों', 'उनकि', 'भि', 'वरग', 'हुअ', 'जेसा', 'नहिं']
stopwords_en = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
# punctuations = ['nn','n', '।','/', '`', '+', '\', '"', '?', '▁(', '$', '@', '[', '_', "'", '!', ',', ':', '^', '|', ']', '=', '%', '&', '.', ')', '(', '#', '*', '', ';', '-', '}','|','"']

# punctuations = ['(', ')', '?', ':', ';', ',', '.', '!', '/', '"', "'"]

punctuations = [".", ",", "!", "?", ":", ";", "-", "—", "(", ")", "[", "]", "{", "}", "/", "\\", "|", "@", "#", "$", "%", "^", "&", "*", "+", "=", "<", ">", "_", "~", "`", "'", "\"", "«", "»", "“", "”", "‘", "’", "‹", "›", "„", "‟", "‛", "‟", "‘", "’", "‚", "‛", "‟", "‹", "›", "〈", "〉", "《", "》", "「", "」", "『", "』", "【", "】"]



to_be_removed = stopwords_hi + punctuations + stopwords_en

In [ ]:
# removing all stopword

for i in range(len(df)):
    df['text'][i]=[ele for ele in df['text'][i] if ele not in (to_be_removed)]
# count_length()
df.tail(10)

,label,text
20173,0,"[ठोक, तू, अच्छा, ना]"
20174,1,"[💥💥💥, गुड, नाइट, जी💥💥💥💥🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂..."
20175,0,"[तेरी, माँ, कसम]"
20176,1,"[जीवन, जीने, के, तरीके, मायने, के, चमत्कार, नह..."
20177,1,"[तीसरा, नाम्, कल्लू]"
20178,0,"[छोटे, क्यु, पहनते, जरूरत]"
20179,1,"[कैसी, सोना, हमसे, दोस्ती, करोगी]"
20180,1,"[पता, चलता, चुनाव, आ, है🙄🙄🙄]"
20181,1,"[खेत, जोते, किसान, भैंस, के, साथो]"
20182,0,"[काश, आगर, इतना, दिमाग, पढाई, लगायें, शायद, जा..."


In [ ]:
# Converting list to string 

# Create a new column called 'processed_text'
df['processed_text'] = ''

# Iterate over each row of the DataFrame
for i in range(len(df)):
    # Join the list of strings in the 'text' column with a space separator
    text_string = ' '.join(df['text'][i])
    # Remove the stopwords from the text string
    text_string = ' '.join([word for word in text_string.split() if word.lower() not in to_be_removed])
    # Update the 'processed_text' column with the processed text string
    df.loc[i, 'processed_text'] = text_string

# # Call the count_length() function (assuming it's defined somewhere)
# count_length()

df.tail(10)


,label,text,processed_text
20173,0,"[ठोक, तू, अच्छा, ना]",ठोक तू अच्छा ना
20174,1,"[💥💥💥, गुड, नाइट, जी💥💥💥💥🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂...",💥💥💥 गुड नाइट जी💥💥💥💥🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂...
20175,0,"[तेरी, माँ, कसम]",तेरी माँ कसम
20176,1,"[जीवन, जीने, के, तरीके, मायने, के, चमत्कार, नह...",जीवन जीने के तरीके मायने के चमत्कार नही दुसरे ...
20177,1,"[तीसरा, नाम्, कल्लू]",तीसरा नाम् कल्लू
20178,0,"[छोटे, क्यु, पहनते, जरूरत]",छोटे क्यु पहनते जरूरत
20179,1,"[कैसी, सोना, हमसे, दोस्ती, करोगी]",कैसी सोना हमसे दोस्ती करोगी
20180,1,"[पता, चलता, चुनाव, आ, है🙄🙄🙄]",पता चलता चुनाव आ है🙄🙄🙄
20181,1,"[खेत, जोते, किसान, भैंस, के, साथो]",खेत जोते किसान भैंस के साथो
20182,0,"[काश, आगर, इतना, दिमाग, पढाई, लगायें, शायद, जा...",काश आगर इतना दिमाग पढाई लगायें शायद जाकर डॉक्ट...


In [ ]:
traindf, valdf = train_test_split(df,train_size=0.8)

# Preprocessing Test Dataset

In [ ]:
# Loading the dataset
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/hindi_test.csv')
# df = full_df[["text"]]
tdf = test_df
# df["text"] = df["text"].astype(str)
tdf.head()

,label,text
0,0,मैं ये नही सोच रहा की इसे निकले कैसे मैं ये सो...
1,1,और दिवाली में भी पूरा देश पड़ाका नहीं फोडात
2,1,कुत्ता बिल्ली पाल लेना मगर गलत फहमी कभी नहीं।
3,0,तेरी गांड में प्याज काट देगा गुज्जर भोसड़ी के
4,1,बंगाली साड़ी ऐसे नहीं पहना जाता है दीदी


In [ ]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens
tdf['text'] = tdf['text'].apply(lambda x: tokenization(x))

# removing all stopword

for i in range(len(tdf)):
    tdf['text'][i]=[ele for ele in tdf['text'][i] if ele not in (to_be_removed)]
# count_length()
# tdf.tail(10)

# Converting list to string 

# Create a new column called 'processed_text'
tdf['tprocessed_text'] = ''

# Iterate over each row of the DataFrame
for i in range(len(tdf)):
    # Join the list of strings in the 'text' column with a space separator
    text_string = ' '.join(tdf['text'][i])
    # Remove the stopwords from the text string
    text_string = ' '.join([word for word in text_string.split() if word.lower() not in to_be_removed])
    # Update the 'processed_text' column with the processed text string
    tdf.loc[i, 'tprocessed_text'] = text_string

# Call the count_length() function (assuming it's defined somewhere)
# count_length()

tdf.tail(10)


,label,text,tprocessed_text
6718,0,"[चूतियो, कमी, नहीं, बढ़कर, चुतिय]",चूतियो कमी नहीं बढ़कर चुतिय
6719,0,"[प्यार❤, व्यार, 💔साला, सब, धोखा, 😡है, बिना, जि...",प्यार❤ व्यार 💔साला सब धोखा 😡है बिना जिंदगी 🌎जी...
6720,0,"[नंगी, होकर, वीडियो, बनाओ, अच्छी, लगोगी]",नंगी होकर वीडियो बनाओ अच्छी लगोगी
6721,0,"[सही, तरीके, कपड़े, दिखाने, के, मेरे, पास, कपड़ा]",सही तरीके कपड़े दिखाने के मेरे पास कपड़ा
6722,1,"[बीजेपी, देन, मंदिर, सब, मंदिर, रुकवाने, कसर, ...",बीजेपी देन मंदिर सब मंदिर रुकवाने कसर नहीं छोड...
6723,0,"[आरे, पागल, लवडे, समज, आये, वो, ना]",आरे पागल लवडे समज आये वो ना
6724,0,"[शेर, नज़र, अंदाज़, ।, कमजोरी, मत, समझ, लेना, तु...",शेर नज़र अंदाज़ । कमजोरी मत समझ लेना तुम्हारे कौ...
6725,0,"[लाल, चड्डी, के, लाल, होगा]",लाल चड्डी के लाल होगा
6726,1,[],
6727,0,"[पोरीच्या, फोटो, काय, महाराजांचे, नाटक, रे, मा...",पोरीच्या फोटो काय महाराजांचे नाटक रे मादरचोत


# IMPLEMENTATION OF LSTM

In [ ]:
def Tokenize(column, seq_len):
    ##Create vocabulary of words from column
    corpus = [word for text in df['processed_text'] for word in text.split()]
    count_words = Counter(corpus)
    sorted_words = count_words.most_common()
    vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

    ##Tokenize the columns text using the vocabulary
    text_int = []
    for text in column:
        r = [vocab_to_int[word] for word in text.split()]
        text_int.append(r)
    ##Add padding to tokens
    features = np.zeros((len(text_int), seq_len), dtype = int)
    for i, review in enumerate(text_int):
        if len(review) <= seq_len:
            zeros = list(np.zeros(seq_len - len(review)))
            new = zeros + review
        else:
            new = review[: seq_len]
        features[i, :] = np.array(new)

    return sorted_words, features



In [ ]:
max_len=100
vocabulary, tokenized_column = Tokenize(df["processed_text"], max_len)

In [ ]:
tokenized_column[10]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1248,
       1600])

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:

X = df['processed_text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train)

In [ ]:
Word2vec_train_data = list(map(lambda x: x.split(), X_train))
EMBEDDING_DIM = 200


In [ ]:
from gensim.models import Word2Vec

In [ ]:
word2vec_model = Word2Vec(Word2vec_train_data, vector_size=EMBEDDING_DIM)

In [ ]:
print(f"Vocabulary size: {len(vocabulary) + 1}")

Vocabulary size: 32408


In [ ]:
VOCAB_SIZE = len(vocabulary) + 1 #+1 for the padding


In [ ]:
#define empty embedding matrix
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))
    
#fill the embedding matrix with the pre trained values from word2vec
#    corresponding to word (string), token (number associated to the word)
for word, token in vocabulary:
    if word2vec_model.wv.__contains__(word):
        embedding_matrix[token] = word2vec_model.wv.__getitem__(word)

print("Embedding Matrix Shape:", embedding_matrix.shape)

Embedding Matrix Shape: (32408, 200)


In [ ]:
# 
X = tokenized_column
y = df['label'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train, random_state=42)

In [ ]:
(unique, counts) = np.unique(y_train, return_counts=True)
np.asarray((unique, counts)).T

array([[   0, 7579],
       [   1, 6952]])

In [ ]:
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
#Tokenization for LSTM
from collections import Counter
import torch.nn as nn

In [ ]:
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test)) #
valid_data = TensorDataset(torch.from_numpy(X_valid), torch.from_numpy(y_valid))
BATCH_SIZE = 32
train_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True) 
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True) #

In [ ]:
NUM_CLASSES = 5 #We are dealing with a multiclass classification of 5 classes
HIDDEN_DIM = 100 #number of neurons of the internal state (internal neural network in the LSTM)
LSTM_LAYERS = 1 #Number of stacked LSTM layers

LR = 3e-4 #Learning rate
DROPOUT = 0.5 #LSTM Dropout
BIDIRECTIONAL = True #Boolean value to choose if to use a bidirectional LSTM or not
EPOCHS = 10 #Number of training epoch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
class BiLSTM_Sentiment_Classifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes, lstm_layers, bidirectional,batch_size, dropout):
        super(BiLSTM_Sentiment_Classifier,self).__init__()
        
        self.lstm_layers = lstm_layers
        self.num_directions = 2 if bidirectional else 1
        self.hidden_dim = hidden_dim
        self.num_classes = num_classes
        self.batch_size = batch_size
        

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers=lstm_layers,
                            dropout=dropout,
                            bidirectional=bidirectional,
                            batch_first=True)

        self.fc = nn.Linear(hidden_dim*self.num_directions, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, hidden):
        self.batch_size = x.size(0)
        ##EMBEDDING LAYER
        embedded = self.embedding(x)
        #LSTM LAYERS
        out, hidden = self.lstm(embedded, hidden)
        #Extract only the hidden state from the last LSTM cell
        out = out[:,-1,:]
        #FULLY CONNECTED LAYERS
        out = self.fc(out)
        out = self.softmax(out)

        return out, hidden

    def init_hidden(self, batch_size):
        #Initialization of the LSTM hidden and cell states
        h0 = torch.zeros((self.lstm_layers*self.num_directions, batch_size, self.hidden_dim)).detach().to(DEVICE)
        c0 = torch.zeros((self.lstm_layers*self.num_directions, batch_size, self.hidden_dim)).detach().to(DEVICE)
        hidden = (h0, c0)
        return hidden
model = BiLSTM_Sentiment_Classifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM,NUM_CLASSES, LSTM_LAYERS,BIDIRECTIONAL, BATCH_SIZE, DROPOUT)
model = model.to(DEVICE)

#Initialize embedding with the previously defined embedding matrix
model.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
#Allow the embedding matrix to be fined tuned to better adapt to out dataset and get higher accuracy
model.embedding.weight.requires_grad=True

print(model)

BiLSTM_Sentiment_Classifier(
  (embedding): Embedding(32408, 200)
  (lstm): LSTM(200, 100, batch_first=True, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=200, out_features=5, bias=True)
  (softmax): LogSoftmax(dim=1)
)


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
criterion = nn.NLLLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay = 5e-6)

# Below cell is for trainig if dont want to train dont run it

In [ ]:
total_step = len(train_loader)
total_step_val = len(valid_loader)

early_stopping_patience = 4
early_stopping_counter = 0

valid_acc_max = 0 # Initialize best accuracy top 0

for e in range(EPOCHS):

    #lists to host the train and validation losses of every batch for each epoch
    train_loss, valid_loss  = [], []
    #lists to host the train and validation accuracy of every batch for each epoch
    train_acc, valid_acc  = [], []

    #lists to host the train and validation predictions of every batch for each epoch
    y_train_list, y_val_list = [], []

    #initalize number of total and correctly classified texts during training and validation
    correct, correct_val = 0, 0
    total, total_val = 0, 0
    running_loss, running_loss_val = 0, 0


    ####TRAINING LOOP####

    model.train()

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE) #load features and targets in device

        h = model.init_hidden(labels.size(0))

        model.zero_grad() #reset gradients 

        output, h = model(inputs,h) #get output and hidden states from LSTM network
        
        loss = criterion(output, labels)
        loss.backward()
        
        running_loss += loss.item()
        
        optimizer.step()

        y_pred_train = torch.argmax(output, dim=1) #get tensor of predicted values on the training set
        y_train_list.extend(y_pred_train.squeeze().tolist()) #transform tensor to list and the values to the list
        
        correct += torch.sum(y_pred_train==labels).item() #count correctly classified texts per batch
        total += labels.size(0) #count total texts per batch

    train_loss.append(running_loss / total_step)
    train_acc.append(100 * correct / total)

    ####VALIDATION LOOP####
    
    with torch.no_grad():
        
        model.eval()
        
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            val_h = model.init_hidden(labels.size(0))

            output, val_h = model(inputs, val_h)

            val_loss = criterion(output, labels)
            running_loss_val += val_loss.item()

            y_pred_val = torch.argmax(output, dim=1)
            y_val_list.extend(y_pred_val.squeeze().tolist())

            correct_val += torch.sum(y_pred_val==labels).item()
            total_val += labels.size(0)

        valid_loss.append(running_loss_val / total_step_val)
        valid_acc.append(100 * correct_val / total_val)

    #Save model if validation accuracy increases
    if np.mean(valid_acc) >= valid_acc_max:
        torch.save(model.state_dict(), './lstm_model.pth')
        print(f'Epoch {e+1}:Validation accuracy increased ({valid_acc_max:.6f} --> {np.mean(valid_acc):.6f}).  Saving model ...')
        valid_acc_max = np.mean(valid_acc)
        early_stopping_counter=0 #reset counter if validation accuracy increases
    else:
        print(f'Epoch {e+1}:Validation accuracy did not increase')
        early_stopping_counter+=1 #increase counter if validation accuracy does not increase
        
    if early_stopping_counter > early_stopping_patience:
        print('Early stopped at epoch :', e+1)
        break
    
    print(f'\tTrain_loss : {np.mean(train_loss):.4f} Val_loss : {np.mean(valid_loss):.4f}')
    print(f'\tTrain_acc : {np.mean(train_acc):.3f}% Val_acc : {np.mean(valid_acc):.3f}%')

Epoch 1:Validation accuracy increased (0.000000 --> 73.312500).  Saving model ...
	Train_loss : 0.7472 Val_loss : 0.6180
	Train_acc : 55.789% Val_acc : 73.312%
Epoch 2:Validation accuracy increased (73.312500 --> 79.000000).  Saving model ...
	Train_loss : 0.4364 Val_loss : 0.4484
	Train_acc : 82.358% Val_acc : 79.000%
Epoch 3:Validation accuracy increased (79.000000 --> 80.437500).  Saving model ...
	Train_loss : 0.2279 Val_loss : 0.4742
	Train_acc : 91.795% Val_acc : 80.438%
Epoch 4:Validation accuracy did not increase
	Train_loss : 0.1351 Val_loss : 0.4992
	Train_acc : 95.457% Val_acc : 79.312%
Epoch 5:Validation accuracy did not increase
	Train_loss : 0.0875 Val_loss : 0.5679
	Train_acc : 97.212% Val_acc : 79.750%
Epoch 6:Validation accuracy did not increase
	Train_loss : 0.0632 Val_loss : 0.6319
	Train_acc : 98.259% Val_acc : 78.875%
Epoch 7:Validation accuracy did not increase
	Train_loss : 0.0503 Val_loss : 0.7072
	Train_acc : 98.596% Val_acc : 79.375%
Epoch 8:Validation accurac

# Loading Model

In [ ]:
model = BiLSTM_Sentiment_Classifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM,NUM_CLASSES, LSTM_LAYERS,BIDIRECTIONAL, BATCH_SIZE, DROPOUT)
model = model.to(DEVICE)
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/lstm_model.pth'))

<All keys matched successfully>

# Evaluating our model




In [ ]:
model.eval()
y_pred_list = []
y_test_list = []
for inputs, labels in test_loader:
    inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
    test_h = model.init_hidden(labels.size(0))

    output, val_h = model(inputs, test_h)
    y_pred_test = torch.argmax(output, dim=1)
    y_pred_list.extend(y_pred_test.squeeze().tolist())
    y_test_list.extend(labels.squeeze().tolist())

# Printing Accuracy Report of Trained Model




In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print('Classification Report for LSTM :\n', classification_report(y_test_list, y_pred_list))

Classification Report for LSTM :
               precision    recall  f1-score   support

           0       0.84      0.78      0.81      2102
           1       0.78      0.84      0.81      1930

    accuracy                           0.81      4032
   macro avg       0.81      0.81      0.81      4032
weighted avg       0.81      0.81      0.81      4032



In [ ]:
confusion_matrix(y_test_list,y_pred_list)

array([[1648,  454],
       [ 315, 1615]])

# Testing for given Test.csv File




In [ ]:
def Tokenize(column, seq_len):
    ##Create vocabulary of words from column
    corpus = [word for text in tdf['tprocessed_text'] for word in text.split()]
    count_words = Counter(corpus)
    sorted_words = count_words.most_common()
    vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

    ##Tokenize the columns text using the vocabulary
    text_int = []
    for text in column:
        r = [vocab_to_int[word] for word in text.split()]
        text_int.append(r)
    ##Add padding to tokens
    features = np.zeros((len(text_int), seq_len), dtype = int)
    for i, review in enumerate(text_int):
        if len(review) <= seq_len:
            zeros = list(np.zeros(seq_len - len(review)))
            new = zeros + review
        else:
            new = review[: seq_len]
        features[i, :] = np.array(new)

    return sorted_words, features



In [ ]:
X = tdf['tprocessed_text']
y = tdf['label']

vocabulary, tokenized_column = Tokenize(tdf["tprocessed_text"], max_len)

X = tokenized_column
y = tdf['label'].values

test_data = TensorDataset(torch.from_numpy(X), torch.from_numpy(y))
test_loader = DataLoader(test_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True) #

model.eval()
y_pred_list = []
y_test_list = []
for inputs, labels in test_loader:
    inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
    test_h = model.init_hidden(labels.size(0))

    output, val_h = model(inputs, test_h)
    y_pred_test = torch.argmax(output, dim=1)
    y_pred_list.extend(y_pred_test.squeeze().tolist())
    y_test_list.extend(labels.squeeze().tolist())

# Printing Classification report of Test Dataset

In [ ]:
print('Classification Report for LSTM :\n', classification_report(y_test_list, y_pred_list))

Classification Report for LSTM :
               precision    recall  f1-score   support

           0       0.94      0.90      0.92      1051
           1       0.89      0.94      0.92       933

    accuracy                           0.92      1984
   macro avg       0.92      0.92      0.92      1984
weighted avg       0.92      0.92      0.92      1984



In [ ]:
def print_intersection_points(train_df, val_df, new_testdf):
  train_com_count = 0
  val_com_count = 0
  for sentence in train_df['text']:
    for new_sentence in new_testdf['text']:
      if sentence==new_sentence:
        train_com_count += 1

  for sentence in val_df['text']:
    for new_sentence in new_testdf['text']:
      if sentence==new_sentence:
        val_com_count += 1
  
  # print("Intersection points for dataset used for testing and training ", modelname)
  print("Common points with training dataset: ",train_com_count)
  print("Common points with validation dataset: ",val_com_count)

print_intersection_points(traindf, valdf, test_df)


Common points with training dataset:  166
Common points with validation dataset:  56
